# Agent Types

Toda a ideia de utilizarmos agente é termos um sistema que saiba que possue ferramntas diferentes em mãos para utilizar e aumentar suas capacidades e as utilize nos momentos adequados. Para que isso seja possível, a OpenAI fez diversos processos de fine tunning em seus modelos, de forma que eles consiga gerar outputs paronizadas para a utilização das ferramentas. Entretanto, muito desse processo pode ser feito, ou melhorado, através de Engenharia de Prompts, e por isso, diversos tipos de agentes (Agent Types) foram criados na literatura e no framework LangChain. Cada um tenta otimizar a utilização das ferramentas e vamos falar sobre os dois mais utilizados: Toll Calling e ReAct. Para verificar todos, você pode acessar a página: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/.

## Tool calling agent

Tool calling permite que um modelo detecte quando uma ou mais ferramentas devem ser acionadas e responda com os dados que devem ser passados para essas ferramentas. Em uma chamada de API, você pode descrever ferramentas e fazer com que o modelo escolha inteligentemente gerar um objeto estruturado, como JSON, contendo argumentos para acionar essas ferramentas. O objetivo das APIs de ferramentas é retornar chamadas de ferramentas válidas e úteis de forma mais confiável do que o que pode ser feito usando uma API genérica de conclusão de texto ou chat.

Podemos aproveitar essa saída estruturada, combinada com o fato de que você pode vincular várias ferramentas a um modelo de chat que chama ferramentas e permitir que o modelo escolha qual chamar, para criar um agente que repetidamente chama ferramentas e recebe resultados até que uma consulta seja resolvida.

Esta é uma versão mais generalizada do agente de ferramentas do OpenAI, que foi projetada para o estilo específico de chamada de ferramentas da OpenAI. Ela usa a interface ToolCall do LangChain para suportar uma gama mais ampla de implementações de provedores, como Anthropic, Google Gemini e Mistral, além do OpenAI.

In [1]:
from langchain_experimental.tools import PythonAstREPLTool

tools = [PythonAstREPLTool()]

In [2]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

system_message = '''Você é um assistente gentil e amigável.
Certifique-se de usar a ferramenta PythonAstREPLTool para auxiliar a responder as perguntas'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('placeholder', '{chat_memory}'),
    ('user', '{input}'),
    ('placeholder', '{agent_scratchpad}')
])

chat = ChatOpenAI(model='gpt-4o-mini')
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [4]:
agent_executor.invoke({'input': 'Qual o décimo elemento da sequência de fibonacci?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a\n\nfibonacci(10)'}`


55O décimo elemento da sequência de Fibonacci é 55.

> Finished chain.


{'input': 'Qual o décimo elemento da sequência de fibonacci?',
 'output': 'O décimo elemento da sequência de Fibonacci é 55.'}

In [5]:
agent_executor.invoke({'input': 'Quantas letras tem a palavra LangChain?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "len('LangChain')"}`


9A palavra "LangChain" tem 9 letras.

> Finished chain.


{'input': 'Quantas letras tem a palavra LangChain?',
 'output': 'A palavra "LangChain" tem 9 letras.'}

In [6]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

system_message = '''Você é um agente projetado para escrever e executar códigos Python para responder perguntas.
Você tem acesso a um REPL Python, que pode usar para executar o código Python.
Se encontrar um erro, depure o código e tente novamente.
Use apenas a saída do seu código para responder à pergunta.
Você pode conhecer a resposta sem executar nenhum código, mas deve ainda assim executar o código para obter a resposta.
Se não parecer possível escrever código para responder à pergunta, simplesmente retorne "Não sei" como a resposta.'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('placeholder', '{chat_memory}'),
    ('user', '{input}'),
    ('placeholder', '{agent_scratchpad}')
])

chat = ChatOpenAI(model='gpt-4o-mini')
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({'input': 'Qual o décimo elemento da sequência de fibonacci?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a\n\nfibonacci(10)'}`


55O décimo elemento da sequência de Fibonacci é 55.

> Finished chain.


{'input': 'Qual o décimo elemento da sequência de fibonacci?',
 'output': 'O décimo elemento da sequência de Fibonacci é 55.'}

## ReAct agent (Reason + Act)

É uma técnica de Engenharia de Prompts criada para permitir que os LLMs interajam com ferramentas externas para recuperar informações adicionais que levam a respostas mais confiáveis e factuais.


In [8]:
from langchain.agents import create_react_agent

https://smith.langchain.com/hub/

In [9]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [10]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [12]:
chat = ChatOpenAI(model='gpt-4o-mini')
agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [13]:
agent_executor.invoke({'input': 'Qual o décimo elemento da sequência de fibonacci?'})



> Entering new AgentExecutor chain...
A sequência de Fibonacci é definida como uma sequência onde cada número é a soma dos dois números anteriores, começando com 0 e 1. Os primeiros elementos da sequência são: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34. 

Para encontrar o décimo elemento da sequência, é mais eficiente usar uma abordagem programática que calcula os elementos. Assim, vou implementá-la.

Action: python_repl_ast  
Action Input: 
```python
def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

fibonacci(10)
```  55I now know the final answer  
Final Answer: 55

> Finished chain.


{'input': 'Qual o décimo elemento da sequência de fibonacci?', 'output': '55'}

In [ ]:
'''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''